In [1]:
!pip install PyPDF2 transformers sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
model_name = "microsoft/DialoGPT-medium"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [18]:
def chat_with_pdf(pdf_text):
    conversation_history = ""
    context_chunk_size = 500  # Define the size of each context chunk

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break

        # Append user input to conversation history
        conversation_history += f"Human: {user_input}\n"

        # Add a relevant chunk of PDF text to the conversation history
        if len(pdf_text) > context_chunk_size:
            pdf_chunk = pdf_text[:context_chunk_size]
            pdf_text = pdf_text[context_chunk_size:]  # Remove the used chunk from the text
        else:
            pdf_chunk = pdf_text

        conversation_history += f"PDF: {pdf_chunk}\n"

        # Print conversation history for debugging
        print("Conversation History:\n", conversation_history)

        # Encode the input
        inputs = tokenizer.encode(conversation_history + "Assistant:", return_tensors="pt")

        # Generate a response
        attention_mask = (inputs != tokenizer.pad_token_id) if tokenizer.pad_token_id is not None else None

        outputs = model.generate(
            inputs,
            max_length=1000,
            pad_token_id=tokenizer.eos_token_id,
            attention_mask=attention_mask
        )

        # Decode the response
        response = tokenizer.decode(outputs[:, inputs.shape[-1]:][0], skip_special_tokens=True)
        print(f"Assistant: {response}")

        # Append response to conversation history
        conversation_history += f"Assistant: {response}\n"


In [19]:
# Start chat with the extracted PDF text
chat_with_pdf(text)


KeyboardInterrupt: Interrupted by user